ELLpack storage format for sparse matrices: Store (both in column major order)
- Matrix: nonzeros grouped by rows + padding
- Matrix: indices of each (equal padding as above)

Thus, the approach to SpMV is as follow:
- Assin a thread → each input row
- Loops over input row
- Updates output

## Setup

In [ ]:
!pip install wurlitzer
!pip install Ninja
!pip install dill
import os,math,sys,torch,re,numpy as np
from types import SimpleNamespace as ns
from collections import namedtuple
# from utils import show_img,load_cuda,cuda_begin,cdiv

# Utils

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.cpp_extension import load_inline

import os,math,sys,torch,re,numpy as np
from types import SimpleNamespace as ns
from collections import namedtuple

np.set_printoptions(precision=2, linewidth=140)
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)

def show_img(x, figsize=(4,3), **kwargs):
    "Display HW or CHW format image `x`"
    plt.figure(figsize=figsize)
    plt.axis('off')
    if len(x.shape)==3: x = x.permute(1,2,0)  # CHW -> HWC
    plt.imshow(x.cpu(), **kwargs)

cuda_begin = r'''
#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)
#define CUDA_ERR(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}
__host__ __device__ inline unsigned int cdiv(unsigned int a, unsigned int b) { return (a+b-1)/b;}
'''

def load_cuda(cuda_src, cpp_src, funcs, opt=True, verbose=False, name=None):
    "Simple wrapper for torch.utils.cpp_extension.load_inline"
    if name is None: name = funcs[0]
    # flags = "-O3 -Xptxas -O3 -Xcompiler -O3" if opt else "-O0 -Xptxas -O0 -Xcompiler -O0"
    return load_inline(cuda_sources=[cuda_src], cpp_sources=[cpp_src], functions=funcs, verbose=verbose, name=name)

def cdiv(a,b):
    "Int ceiling division of `a` over `b`"
    return (a+b-1)//b

def get_sig(fname, src):
    res = re.findall(rf'^(.+\s+{fname}\(.*?\))\s*{{?\s*$', src, re.MULTILINE)
    return res[0]+';' if res else None

dim3 = namedtuple('dim3', ['x','y','z'], defaults=(1,1))



In [ ]:
%load_ext wurlitzer

# Numba Version in CUDA format



In [53]:
from numba import cuda
import numpy as np
import math
import torch

@cuda.jit
def kogge_kernel(nonzeros, indices, v, product):
    """
    Kernel for computing sparse matrix-vector multiplication (SpMV)
    using the ELLPACK format.

    Parameters:
    - nonzeros: 2D array with nonzero values of the matrix, column-major.
    - indices: Corresponding column indices for nonzeros, same shape as nonzeros.
    - v: Dense vector for multiplication.
    - product: Result vector.
    """
    row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    if row < product.size:  # Check if within bounds
        sum = 0.0
        for i in range(nonzeros.shape[1]):  # Iterate over elements in row
            col = indices[row, i]
            if col < v.size:  # Valid column index guard
                sum += nonzeros[row, i] * v[col]
        product[row] = sum

def run_spmv(nonzeros, indices, v, block_size=256):
    """
    Function to perform SpMV using CUDA.

    Parameters:
    - nonzeros: 2D numpy array with nonzero values of the matrix, column-major.
    - indices: Corresponding column indices for nonzeros, same shape as nonzeros.
    - v: Dense vector for multiplication (as a numpy array).
    - block_size: Number of threads per block.
    """
    # Ensure inputs are suitable numpy arrays
    nonzeros = np.ascontiguousarray(nonzeros)
    indices = np.ascontiguousarray(indices)
    v = np.ascontiguousarray(v)

    num_rows = nonzeros.shape[0]

    product = np.zeros(num_rows, dtype=v.dtype)

    # Calculate grid dimensions
    block_no = math.ceil(num_rows / block_size)

    # Setting up device arrays
    d_nonzeros = cuda.to_device(nonzeros)
    d_indices = cuda.to_device(indices)
    d_v = cuda.to_device(v)
    d_product = cuda.to_device(product)

    # Launch kernel
    kogge_kernel[block_no, block_size](d_nonzeros, d_indices, d_v, d_product)

    # Copy result back to host
    product = d_product.copy_to_host()

    return product


nonzeros = torch.tensor([[5, 0], [8, 0], [3, 6]])
indices = torch.tensor([[0, 0], [1, 0], [2, 2]])

v = torch.tensor([1, 2, 3])

result = run_spmv(nonzeros, indices, v)
print(result)  # Output of the SpMV


[ 5 16 27]


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [54]:
import torch

def get_sparse_unformatted_matrix(rows=10, cols=10):
    dense_matrix = torch.zeros((rows, cols))

    # Assume we want ~20% of the matrix to be non-zero
    num_nonzeros = int(0.2 * rows * cols)

    # Randomly choose indices for non-zero elements
    # Ensure unique positions if randomness allows duplicates that you don't want
    row_indices = torch.randint(0, rows, (num_nonzeros,))
    col_indices = torch.randint(0, cols, (num_nonzeros,))

    # Populate the selected positions with random non-zero values
    for i in range(num_nonzeros):
        # You can adjust the range of random values as needed
        dense_matrix[row_indices[i], col_indices[i]] = torch.rand(1)
    return dense_matrix
def to_column_major(tensor):
    """
    Convert a 2D tensor to column-major order by returning a transposed,
    contiguous version of the tensor.

    Note: This doesn't change the physical layout in memory to column-major
    but returns a view that simulates the behavior.

    Parameters:
    tensor (torch.Tensor): A 2D tensor.

    Returns:
    torch.Tensor: A tensor transposed and made contiguous to simulate column-major order.
    """
    # Transpose to change row-major to column-major indexing
    transposed = tensor.t()
    # Ensure the transposed tensor is contiguous
    column_major_tensor = transposed.contiguous()
    return column_major_tensor


def dense_to_ell_format(dense_tensor):
    """
    Convert a dense matrix (torch.Tensor) into ELL format.

    Parameters:
    dense_tensor (torch.Tensor): The dense matrix.

    Returns:
    (values, indices): A tuple of two 2D tensors - values and indices in ELL format.
    """
    if not torch.is_tensor(dense_tensor) or dense_tensor.is_sparse:
        raise ValueError("Input tensor must be a dense torch.Tensor")

    # Convert the dense tensor to sparse COO tensor
    sparse_coo_tensor = dense_tensor.to_sparse()

    # Ensure the input tensor is in COO format and coalesced
    sparse_coo_tensor = sparse_coo_tensor.coalesce()

    # Get the indices and values of the non-zero elements
    indices = sparse_coo_tensor.indices()
    values = sparse_coo_tensor.values()

    num_rows, num_cols = dense_tensor.shape

    # Count non-zeros in each row
    row_nnz = torch.zeros(num_rows, dtype=torch.int64)
    row_indices = indices[0]
    for row_index in row_indices:
        row_nnz[row_index] += 1

    # Find the maximum number of non-zero elements in any row
    max_nnz_per_row = torch.max(row_nnz)

    # Initialize tensors for ELL format (using padding where necessary)
    ell_values = torch.zeros((num_rows, max_nnz_per_row), dtype=values.dtype)
    ell_indices = torch.full((num_rows, max_nnz_per_row), num_cols, dtype=torch.int64)  # Use `num_cols` as padding value

    # Populate the ELL format tensors
    current_count = torch.zeros(num_rows, dtype=torch.int64)
    for row, col in indices.t():
        idx = current_count[row]
        ell_indices[row, idx] = col
        ell_values[row, idx] = values[current_count[row]]
        current_count[row] += 1

    return to_column_major(ell_values), to_column_major(ell_indices)


sparse = get_sparse_unformatted_matrix()
sparse_matrix  =dense_to_ell_format(sparse)

In [55]:
type(sparse_matrix)

tuple

In [56]:
sparse_matrix[0].shape

torch.Size([4, 10])

In [57]:
sparse_matrix[1]

tensor([[ 3,  2,  5,  7,  4,  3,  2,  3,  2, 10],
        [ 5, 10, 10,  9,  5,  5,  3,  5, 10, 10],
        [ 6, 10, 10, 10, 10,  6, 10,  7, 10, 10],
        [ 7, 10, 10, 10, 10, 10, 10, 10, 10, 10]])

In [58]:
vector = torch.rand([10])

In [61]:
sparse @ vector

tensor([1.0549, 0.0187, 0.3387, 0.8842, 0.8266, 0.8998, 0.1620, 1.5696, 0.0932,
        0.0000])